In [4]:
import re
import time
import copy
import math
import json
import yaml
import random
import itertools
import dicttoxml, xmltodict
from enum import Enum

from pprint import pprint as pp
from functools import wraps, partial

## Motivation

Combine data + logic in one place using language syntax
Express high-level abstraction from problem domain: Student, Employee, Schedule, Vehicle, Invoice, Item, Patient, etc
Better structuring by inheritance: common data and logic is placed in one place (base class)

In [3]:
class Drone:
    
    class MoveDirection(Enum):
        FORWARD = 1
        BACKWARD = 2
        UP = 3
        DOWN = 4
        
#     MAX_SPEED = 100
#     MAX_ALTITUDE = 1000
#     MAX_PAYLOAD = 10

#     model = None
#     current_speed = 0
#     current_payload = 0
#     current_altitude = 0
    
    def __init__(self, model, current_payload):
        self.model = model
        self.current_payload = current_payload

    def _move(self, speed, direction):
        ...

    def move_backward(self, speed):
        print(self.model)
        self._move(speed, self.MoveDirection.BACKWARD)
        
    def move_down(self, speed):
        if self.current_altitude == 0:
            print ('ERROR: Can\'t move down')
        self._move(speed, self.MoveDirection.DOWN)
        
        
dr1 = Drone('XS-100', 10)
dr2 = Drone('XS-200', 20)
# dr3 = Drone()


dr1.move_backward(10)
dr2.move_backward(10)

Drone.move_backward(dr1, 10)   # -> dr1.move_backward(10)
Drone.move_backward(dr2, 10)   # -> dr2.move_backward(10)



XS-100
XS-200
XS-100
XS-200


#### alternative (C-style)

In [1]:
DRONE_MOVE_FORWARD = 1
DRONE_MOVE_BACKWARD = 2
DRONE_MOVE_UP = 3
DRONE_MOVE_DOWN = 4

DRONE_MAX_ALTITUDE = 1000

drone = {
    'model': '',
    'current_payload': 0,
    'current_speed': 0,
    'current_payload': 0,
    'current_altitude': 0
}

def drone_init(drone, model, payload):
    drone['model'] = model
    drone['current_payload'] = payload

def drone_move(drone, speed, direction):
    pass
    
def drone_move_up(drone, speed):
    print(drone['model'])
    if drone['current_altitude'] == DRONE_MAX_ALTITUDE:
        print ('ERROR: Can\'t move up')
    drone_move(drone, speed, DRONE_MOVE_UP)

def drone_move_down(drone, speed):
    print(drone['model'])
    if drone['current_altitude'] == 0:
        print ('ERROR: Can\'t move down')
    drone_move(drone, speed, DRONE_MOVE_DOWN)
    
    
#####
dr1 = drone.copy()
drone_init(dr1, 'XS-100', 10)

dr2 = drone.copy()
drone_init(dr2, 'XS-200', 20)

drone_move_up(dr1, 10)
drone_move_down(dr2, 5)

XS-100
XS-200
ERROR: Can't move down



# Basics

<img src="https://i.imgur.com/x2LZO0V.png">

We can manually assign attributes using monkey patching so that all instances have the same (!) set of attributes

In [2]:
# Monkey patching
class Drone:
    pass

dr1 = Drone()
dr1.model = 'XS-100'
dr1.payload = 10

dr2 = Drone()
dr2.model = 'XS-200'
dr2.payload = 20
...

drones = []
drones.append(dr1)
drones.append(dr2)

for drone in drones:
    print(drone.model, drone.payload)
#     drone.move_up(...)

XS-100 10
XS-200 20


 But it is more convenient to do it 1 place if we know, that it will be called for each (!) new instance.
 This place is magic method __init__, used as constructor where object attributes are assigned

In [3]:
class Drone:
    
    class MoveDirection(Enum):
        FORWARD = 1
        BACKWARD = 2
        UP = 3
        DOWN = 4
    
    def __init__(self, model, payload):
        print('INIT', id(self))
        self.model = model
        self.payload = payload
        
    def _move(self, speed, direction):
        ...
        
    def move_backward(self, speed):
        print(self.model)
        self._move(speed, self.MoveDirection.BACKWARD)
        
    def move_down(self, speed):
        if self.current_altitude == 0:
            print ('ERROR: Can\'t move down')
        self._move(speed, self.MoveDirection.DOWN)



dr1 = Drone('XS-100', 10)
dr2 = Drone('XS-150', 20)

print('Object id:', id(dr1))
dr1.move_backward(10) 
Drone.move_backward(dr1, 10) 


NameError: name 'Enum' is not defined

# Inheritance

In [69]:
class Shape:
    
    NUM_OF_DIMS = 2
   
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    #@staticmethod
    @classmethod
    def distance(cls, x1, y1, x2, y2):
        return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
        
    def distance_from(other):
        return self.distance(self.x, self.y, other.x, other.y)
        
    def render(self):
        print('Hello from render()')

    def scale(self, scale_factor):
        print('Hello from scale()')
        
    def square(self):
        print('Hello from square()')
        

class Circle(Shape):
    def __init__(self, x, y, radius):
        super().__init__(x, y)
        self.radius = radius

        
class Rectangle(Shape):
    def __init__(self, x, y, width, height):
        super().__init__(x, y)
        self.width = width
        self.height = height

        
class Ellipse(Circle):
    def __init__(self, x, y, radius1, radius2):
        super().__init__(x, y, radius1)
        self.radius2 = radius2

        
class Polygon(Rectangle):
    """Polygon class doc"""

    MIN_NUM_OF_POINTS = 3 # constant
    default_number_of_sides = 3 # default value (IMMUTABLE)
    default_sides = (1, 1, 1) # BAD!!!
    
    def __init__(self,  x, y, width, height, **kwargs):
        super().__init__(x, y, width, height)
        self.points = kwargs.get('points')
        self.num_of_points = kwargs.get('num_of_points')
        self.sides = kwargs.get('sides', list(self.default_sides))
        self.default_number_of_sides = 3


c1 = Circle(10, 10, 42)
c1.render()
print(c1.x, c1.y, c1.radius)

r1 = Rectangle(10, 10, 42, 42)
r1.render()
print(r1.x, r1.y, r1.width)

o1 = Ellipse(10, 10, 42, 42)
o1.render()
print(Ellipse.mro())

p1 = Polygon(10, 20, 30, 40, points=[(1,1), (2,2)], num_of_points=2)

Polygon.mro()


print(Shape.distance(0, 0, 10, 10))
print(p1.distance(0, 0, 10, 10))

sh3 = Shape3D(10, 20, 30)
print(sh3.distance(0, 0, 10, 10))

Hello from render()
10 10 42
Hello from render()
10 10 42
Hello from render()
[<class '__main__.Ellipse'>, <class '__main__.Circle'>, <class '__main__.Shape'>, <class 'object'>]
14.142135623730951
14.142135623730951


In [44]:
# dir(Polygon)
dir(p1)
print(p1.__dict__)

p2 = Polygon(0, 0, 10, 20, points=[], num_of_points=0)
print(p2.__dict__)
p2.__dict__['x'] = 42
#p2.x = 43
print(p2.x)

print(Polygon.__dict__)

print(p1.min_num_of_points)

{'x': 10, 'y': 20, 'width': 30, 'height': 40, 'points': [(1, 1), (2, 2)], 'num_of_points': 2}
{'x': 0, 'y': 0, 'width': 10, 'height': 20, 'points': [], 'num_of_points': 0}
42
{'__module__': '__main__', '__doc__': 'Polygon class doc', 'MIN_NUM_OF_POINTS': 3, 'default_number_of_sides': 3, 'default_sides': (1, 1, 1), '__init__': <function Polygon.__init__ at 0x7f3b04381ca0>}


AttributeError: 'Polygon' object has no attribute 'min_num_of_points'

In [52]:
print(p1.__dict__)
p1.default_number_of_sides = 4
print(p1.__dict__)

print(p2.__dict__)
p2.default_number_of_sides = 5
print(p2.__dict__)

p3 = Polygon(10, 20, 30, 40, points=[(1,1), (2,2)], num_of_points=2)
print(p3.default_number_of_sides)

print(Polygon.__dict__)

p1.default_sides[0] = 42
print(p1.default_sides)

print(p2.default_sides)

{'x': 10, 'y': 20, 'width': 30, 'height': 40, 'points': [(1, 1), (2, 2)], 'num_of_points': 2, 'sides': [1, 1, 1]}
{'x': 10, 'y': 20, 'width': 30, 'height': 40, 'points': [(1, 1), (2, 2)], 'num_of_points': 2, 'sides': [1, 1, 1], 'default_number_of_sides': 4}
{'x': 42, 'y': 0, 'width': 10, 'height': 20, 'points': [], 'num_of_points': 0, 'default_number_of_sides': 5}
{'x': 42, 'y': 0, 'width': 10, 'height': 20, 'points': [], 'num_of_points': 0, 'default_number_of_sides': 5}
3
{'__module__': '__main__', '__doc__': 'Polygon class doc', 'MIN_NUM_OF_POINTS': 3, 'default_number_of_sides': 3, 'default_sides': (1, 1, 1), '__init__': <function Polygon.__init__ at 0x7f3b043329d0>}


TypeError: 'tuple' object does not support item assignment

In [53]:
p4 = Polygon(10, 20, 30, 40, points=[(1,1), (2,2)], num_of_points=2)
p4.sides[0] = 42


print(p4.sides)
print(p3.sides)

[42, 1, 1]
[1, 1, 1]


In [28]:
class A:

    def __init__(self):
        super().__init__()
    
    def foo(self):
        
        print('Hello from A.foo()')


class A1(A):
    def __init__(self):
        super().__init__()
        self.attrA1 = 'attrA1'
        
    def foo_a1(self):
        print('foo_a1() called')

        
class A2(A):
    def __init__(self):
        super().__init__()
        self.attrA2 = 'attrA2'

    def foo_a2(self):
        print('foo_a2() called')

        
a1 = A1()
a2 = A2()
print(a1.attrA1)
print(a2.attrA2)
a1.foo()
a2.foo()

a1.foo_a1()
a2.foo_a2()

attrA1
attrA2
Hello from A.foo()
Hello from A.foo()
foo_a1() called
foo_a2() called


### Everyting is object
Magic methods received from object root class

In [87]:
# __dict__
print(dir(object()))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


#### Difference between class and object from attributes point of view
- used for constants
- or for immutable defaults

In [88]:
pp('Obj attrs: ')
pp(a.__dict__)
print()
pp('Class attrs: ')
pp(A.__dict__)


print(a.obj_attr1)
print(a.__dict__['obj_attr1'])
a.__dict__['obj_attr1'] = 42
print(a.obj_attr1)
A.attr1 = 'XYZ'
print(A.__dict__['attr1'])
print(a.attr1)


a = A(1024, 'abc')
b = A(1024, 'abc'.upper().lower())
# c = A(1024, 'ab'+'c')
print(id(a.attr1), id(b.attr1))
print(id(a.obj_attr1), id(b.obj_attr1))
print(id(a.obj_attr2), id(b.obj_attr2))

'Obj attrs: '


NameError: name 'a' is not defined

In [89]:
"""
  Multiple Inheritance introduces diamond problem
"""

class Person:
    def __init__(self, first_name, last_name, **kwargs): 
        self.first_name = first_name 
        self.last_name = last_name 
        

class TeamMember(Person):                 
    def __init__(self, first_name, last_name, **kwargs):
        super().__init__(first_name, last_name, **kwargs)
        self.salary = kwargs.get("salary", 0)
        self.jobtitle = kwargs.get("jobtitle", 'N/A')


class Architect(TeamMember):                 
    def __init__(self, first_name, last_name, **kwargs):
        super().__init__(first_name, last_name, **kwargs)
        self.certificates = kwargs.get("certificates", [])
        self.jobtitle = 'Architect'

        
class TeamLeader(TeamMember):                 
    def __init__(self, first_name, last_name, **kwargs):
        super().__init__(first_name, last_name, **kwargs)
        self.soft_skills = kwargs.get("soft_skills", [])
        self.jobtitle = 'TeamLeader'
        
    def __str__(self):
        return 'TL'

class CTO(TeamLeader, Architect):                 
    def __init__(self, first_name, last_name, **kwargs):
        super().__init__(first_name, last_name, **kwargs)
        self.projects = kwargs.get("projects")
        self.soft_skills += ['Leadership', 'EQ']
        self.certificates += ['ITIL', 'PMA']
        #self.jobtitle = 'CTO'

        
cto = CTO(
    first_name='Jake',
    last_name='Smith',
    salary=250000
)

print(cto.jobtitle)

CTO.mro()

TeamLeader


[__main__.CTO,
 __main__.TeamLeader,
 __main__.Architect,
 __main__.TeamMember,
 __main__.Person,
 object]

#### Multiple inheritance is mainly used to 'mix in' additional behaviour

In [82]:
class JsonMixin:
    
    MAX_PRINT_SYMBOLS = 20
    
    def to_json(self):
        return json.dumps(self.__dict__)
    
    def __str__(self):
        result = super().__str__()
        result += '\n'
        result += 'CLASS: ' + type(self).__name__
        result += '\n'
        obj_str = self.to_json()
        to_display = min(self.MAX_PRINT_SYMBOLS, len(obj_str))
        if to_display > self.MAX_PRINT_SYMBOLS:
            result += obj_str[:to_display] + '...'
        else:
            result += obj_str
        return result
    

class XMLMixin:
    MAX_PRINT_SYMBOLS = 20
    
    def to_xml(self):
        return dicttoxml.dicttoxml(vars(self)).decode()
    
    def __str__(self):
        result = super().__str__()
        result += '\n'
        result += 'CLASS: ' + type(self).__name__
        result += '\n'
        obj_str = self.to_xml()
        to_display = min(self.MAX_PRINT_SYMBOLS, len(obj_str))
        if to_display > self.MAX_PRINT_SYMBOLS:
            result += obj_str[:to_display] + '...'
        else:
            result += obj_str
        return result


class YamlMixin:
    
    MAX_PRINT_SYMBOLS = 20

    def to_yaml(self):
        return yaml.dump(vars(self))
    
    def __str__(self):
        result = super().__str__()
        result += '\n'
        result += 'CLASS: ' + type(self).__name__
        result += '\n'
        obj_str = self.to_yaml()
        to_display = min(self.MAX_PRINT_SYMBOLS, len(obj_str))
        if to_display > self.MAX_PRINT_SYMBOLS:
            result += obj_str[:to_display] + '...'
        else:
            result += obj_str
        return result

In [92]:
class CTO(YamlMixin, TeamLeader, Architect):
    
    def __init__(self, first_name, last_name, **kwargs):
        super().__init__(first_name, last_name, **kwargs)
        self.projects = kwargs.get("projects")
        self.soft_skills += ['Leadership', 'EQ']
        self.certificates += ['ITIL', 'PMA']
        self.jobtitle = 'CTO'
        
#     def __str__(self):
#         return 'CTO'
        
cto = CTO(
    first_name='Jake',
    last_name='Smith',
    salary=250000
)
print(CTO.mro())
#print(cto.to_yaml())
print(str(cto))

[<class '__main__.CTO'>, <class '__main__.YamlMixin'>, <class '__main__.TeamLeader'>, <class '__main__.Architect'>, <class '__main__.TeamMember'>, <class '__main__.Person'>, <class 'object'>]
TL
CLASS: CTO
certificates:
- ITIL
- PMA
first_name: Jake
jobtitle: CTO
last_name: Smith
projects: null
salary: 250000
soft_skills:
- Leadership
- EQ



#### Template pattern (https://bit.ly/3j6Asid)

In [95]:
class SerializedMixin:
    MAX_PRINT_SYMBOLS = 150
    
    def _serialize(self):
        raise NotImplemented('No impemented')
    
    def __str__(self):
        result = super().__str__()
        result += '\n'
        result += 'CLASS: ' + type(self).__name__
        result += '\n'
        obj_str = self._serialize()
        to_display = min(self.MAX_PRINT_SYMBOLS, len(obj_str))
        if to_display < len(obj_str):
            result += obj_str[:to_display] + '...'
        else:
            result += obj_str
        return result

    
class JsonMixin(SerializedMixin):
    MAX_PRINT_SYMBOLS = 42

    def _serialize(self):
        return json.dumps(vars(self))
    

class XMLMixin(SerializedMixin):
    def _serialize(self):
        return dicttoxml.dicttoxml(vars(self)).decode()


class YamlMixin(SerializedMixin):
    def _serialize(self):
        return yaml.dump(vars(self))

In [97]:
class CTO(YamlMixin, TeamLeader, Architect):
    
    def __init__(self, first_name, last_name, **kwargs):
        super().__init__(first_name, last_name, **kwargs)
        self.projects = kwargs.get("projects")
        self.soft_skills += ['Leadership', 'EQ']
        self.certificates += ['ITIL', 'PMA']
        self.jobtitle = 'CTO'

cto = CTO(
    first_name='Jake',
    last_name='Smith',
    salary=250000
)

print(str(cto))

TL
CLASS: CTO
certificates:
- ITIL
- PMA
first_name: Jake
jobtitle: CTO
last_name: Smith
projects: null
salary: 250000
soft_skills:
- Leadership
- EQ



In [98]:
# classmethod vs staticmethod
import math

class Shape:
    NUM_OF_DIMS = 2

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    #@staticmethod
    @classmethod
    def distance(cls, x1, y1, x2, y2):
        return math.sqrt((x1 - x2) **  cls.NUM_OF_DIMS + (y1 - y2) ** cls.NUM_OF_DIMS)


class Shape3D(Shape):
    # NUM_OF_DIMS = 3

    def __init__(self, x=0, y=0):
        super().__init__(x, y)
        self.z = 0

    # @staticmethod
    # # #@classmethod
    # def distance(cls, x1, y1, x2, y2):
    #     return math.sqrt((x1 - x2) **  Shape3D.NUM_OF_DIMS + (y1 - y2) ** Shape3D.NUM_OF_DIMS)

s = Shape(0, 0)
print(s.distance(100, 100, 10, 10))


s2 = Shape3D(0, 0)
print(s2.distance(100, 100, 10, 10))


127.27922061357856
127.27922061357856


# Magic methods/attributes

In [ ]:
# __str__, __init__, __dict__

By overriding magic methods we achieve so called polymorphism: one method -> many implementations

In [100]:
# implementing protocols

class timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time.time()
        return None

    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start) * 1000
        print(self.message.format(elapsed_time))
        
class A():
    def __enter__(self):
        print('__enter__')
    
    def __exit__(self, type, value, traceback):
        print('__exit__')

# with A():
#     raise ValueError('WRONG!')

In [101]:
# overring implicit conversions 

print(a.__str__())
print(str(a))

dir(a)
a.__dir__()

# xxx(a)
# a.__xxx__()

v = 42
print(v + 1)
print(v.__add__(1))

class Employee:
    def __init__(self, first_name=None, last_name=None, email=None):
        self._first_name = first_name
        self._last_name = last_name
        self._email = email
    
    def __bool__(self):
        return bool(self._first_name or \
               self._last_name or \
               self._email)
            
e = Employee()


if e:
    print('Not empty')
    # do some work
    
if e is not None: # None ~= null
    print('Is not None')
    # do some work

NameError: name 'a' is not defined

In [102]:
# overriding operators (+, -, *, /, ...)
class Q:
    def __init__(self, **params):
        self._params = params
    
    def __or__(self, other):
        self._params.update(other._params)
        return self

#     def __and__(self, other):
#         self._params.update(other._params)
#         return self
    
    def __str__(self):
        result = ''
        for k, v in  self._params.items():
            if result:
                result += ' OR '
#             if result:
#                 result += ' OR '
            result += f'{k}={repr(v)}'
        return result

filter = Q()
filter |= Q(first_name='John')
filter |= Q(last_name='Gonzalez')
filter |= Q(stuff=True)
filter |= Q(age=42)

print(filter)

first_name='John' OR last_name='Gonzalez' OR stuff=True OR age=42


In [1]:
# overriding operators (+, -, *, /, ...)
class Q:
    def __init__(self, **params):
        self._params = params
    
    def __or__(self, other):
        self._params.update(other._params)
        return self

#     def __and__(self, other):
#         self._params.update(other._params)
#         return self
    
    def __str__(self):
        result = ''
        for k, v in  self._params.items():
            if result:
                result += ' OR '
#             if result:
#                 result += ' OR '
            result += f'{k}={repr(v)}'
        return result

filter = Q()
filter |= Q(first_name='John')
filter |= Q(last_name='Gonzalez')
filter |= Q(stuff=True)
filter |= Q(age=42)

print(filter)

first_name='John' OR last_name='Gonzalez' OR stuff=True OR age=42


In [105]:
# overriding get/set attributes -> hook access to/from attributes
class A:
    def __getattr__(self, name):
        print('__getattr__')
#         if name in self.__dict__:
#             value = self.__dict__[name]
#         elif name in self.__class__.__dict__:
#             value = self.__class__.__dict__[name]
#         else:
#             value = super().__getattr__(name)                    
#         return value
        return 42
    
    def __setattr__(self, name, value):
        print('__setattr__')
        self.__dict__[name] = value + 1
        
    def __delattr__(self, name):
        if name in self.__dict__:
            del self.__dict__[name]
            
a = A()
a.boo = 42
print(a.boo)


__setattr__
43


In [108]:
# make objects behave like a function: __call__

def foo():
    print('foo')
foo()
foo.__call__()

class A:
    def __call__(self):
        print('__call__')
    
a = A()
a()

foo
foo
__call__


In [2]:
class lazy_object:
    '''
    Class for deferred instantiation of objects.  Init is called
    only when the first attribute is either get or set.
    '''

    def __init__(self, callable, *args, **kw):
        '''
        callable -- Class of objeсt to be instantiated or functionnn to be called
        *args -- arguments to be used when instantiating object
        **kw  -- keywords to be used when instantiating object
        '''
        self.__dict__['callable'] = callable
        self.__dict__['args'] = args
        self.__dict__['kw'] = kw
        self.__dict__['obj'] = None

    def init_obj(self):
        '''
        Instantiate object if not already done
        '''
        if self.obj is None:
            self.__dict__['obj'] = self.callable(*self.args, **self.kw)

    def __getattr__(self, name):
        self.init_obj()
        return getattr(self.obj, name)

    def __setattr__(self, name, value):
        self.init_obj()
        setattr(self.obj, name, value)

    def __len__(self):
        self.init_obj()
        return len(self.obj)

    def __getitem__(self, idx):
        self.init_obj()
        return self.obj[idx]

    def __copy__(self):
        new_copy = lazy_object(self.callable, self.args, self.kw)
        new_copy.__dict__['obj'] = copy.copy(self.obj)
        return new_copy

In [3]:
class A:
    def __init__(self, num_elem):
        self.attr1 = list(range(num_elem))
        
a = lazy_object(A, num_elem=10**8)

print(a)

In [4]:
with timer('Elapsed: {}ms'):
   type(a.attr1)

with timer('Elapsed: {}ms'):
   type(a.attr1)

with timer('Elapsed: {}s'):
   a1 = copy.copy(a) # быстро
   # print(a1)

with timer('Elapsed: {}s'):
    # a1 = copy.deepcopy(a) # долго
    print(a1)

NameError: name 'timer' is not defined

## Data hiding

In [ ]:
class A: # before
    
    def __init__(self, attr1=None, attr2=None):
        self.attr1 = attr1 # protected (client)
        self.attr2 = attr2 # protected (client)

class A:
    
    def __init__(self, attr1=None, attr2=None):
        self._attr1 = attr1 # protected (client)
        self._attr2 = attr2 # protected (client)
        self.__attr3 = attr2 # private (client&child)

    @property
    def attr1(self):
        print('get attr1')
        return self._attr1
        
    @attr1.setter
    def attr1(self, value):
        print('set attr1')
        if value > 0:
            self._attr1 = value
        else:
            raise ValueError('Invalid data')
       
    @property
    def attr2(self):
        print('get attr2')
        return self._attr2

a = A(attr2='abc')
a.attr1 = 42
print(a.attr1)
#a.attr1 = -1
print(a.attr2)
#a.attr2 = 'ABC'
#print(a.__attr3)

print(a.__dict__)
#print(a._A__attr3)
print(a._A__attr3)
a.__attr3 = 'ABC'
print(a.__attr3)
print(a.__dict__)


In [ ]:
class A: # before
    
    def __init__(self, attr1=None, attr2=None):
        self.attr1 = attr1 # protected (client)
        self.attr2 = attr2 # protected (client)

class A:
    
    def __init__(self, attr1=None, attr2=None):
        self._attr1 = attr1 # protected (client)
        self._attr2 = attr2 # protected (client)
        self.__attr3 = attr2 # private (client&child)

    @property
    def attr1(self):
        print('get attr1')
        return self._attr1
        
    @attr1.setter
    def attr1(self, value):
        print('set attr1')
        if value > 0:
            self._attr1 = value
        else:
            raise ValueError('Invalid data')
       
    @property
    def attr2(self):
        print('get attr2')
        return self._attr2

a = A(attr2='abc')
a.attr1 = 42
print(a.attr1)
#a.attr1 = -1
print(a.attr2)
#a.attr2 = 'ABC'
#print(a.__attr3)

print(a.__dict__)
#print(a._A__attr3)
print(a._A__attr3)
a.__attr3 = 'ABC'
print(a.__attr3)
print(a.__dict__)


In [ ]:
a._attr2 = 'ABC'
print(a._attr2)

# Misc topics

### slots

In [ ]:
class A:
    __slots__ = ['attr1', 'attr2']

a = A()
a.attr1 = 42
a.attr2 = '42'
print(a.attr1, a.attr2)

In [ ]:
#a.attr3 = 'abc'

In [ ]:
class A1:
    def __init__(self, attr1, attr2):
        self.attr1 = attr1
        self.attr2 = attr2

a1 = A1(42, '42')
print(a1.attr1, a1.attr2)

In [ ]:
from sys import getsizeof
print(getsizeof(a), a.__slots__, getsizeof(a.__slots__))
print(getsizeof(a1), a1.__dict__, getsizeof(a1.__dict__))

In [ ]:
with timer('Elapsed: {}ms'):
    for _ in range(10**6):
        a = A()
        a.attr1 = 42
        a.attr2 = '42'
        _ = a.attr1, a.attr2

with timer('Elapsed: {}ms'):
    for _ in range(10**6):
        a = A1(42, '42')
        _ = a.attr1, a.attr2


In [ ]:
# NamedTuple()

### Descriptors

In [ ]:
# Technically, descriptor is a class that supports the following methods: __set__[, __get__],__delete__

In [ ]:
class A:
    attr1 = (int, 0)
    attr2 = (str, '')

    def __getattribute__(self, name): # always called
        if name == '__dict__':
            return super().__getattribute__(name)
        obj_attrs = self.__dict__
        cls_attrs = vars(type(self))
        if name not in obj_attrs:
            if name in cls_attrs:
                _, default = cls_attrs[name]
                self.__dict__[name] = default
        return self.__dict__[name]

#     def __getattr__(self, name): # lookup fallback method
#         print('__getattr__', name)
#         obj_attrs = vars(self) # self.__dict__
#         cls_attrs = vars(type(self)) # self.__class__.__dict__
#         if name not in obj_attrs:
#             if name in cls_attrs:
#                 _, default = cls_attrs[name]
#                 self.__dict__[name] = default
#         return self.__dict__[name]
    
    def __setattr__(self, name, value):
        obj_attrs = vars(self)
        cls_attrs = vars(type(self))
        if name in cls_attrs:
            type_, default = cls_attrs[name]
            if isinstance(value, type_):
                self.__dict__[name] = value
            else:
                raise ValueError('Invalid type')

    def __delattr__(self, name):
        del self.__dict__[name]

In [ ]:
a = A()
a1 = A()
print(a.attr1)
print(a.attr2)

a.attr1 = 32
a.attr2 = '42'

a1.attr1 = 33
a1.attr2 = 'xyz'

print(a.attr1, a1.attr1)
print(a.attr2, a1.attr2)


Technically, descriptor is a class that supports the following methods: __set__[, __get__],__delete__

In [ ]:
class Attribute:

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        #print('Retrieving', self.name, id(obj))
        return obj.__dict__[self.name] #self.val

    def __set__(self, obj, val):
        #print('Updating', self.name, id(obj))
        self.val = val
        obj.__dict__[self.name] = val
        
    def __delete__(self, obj):
        #print('Deleting', self.name, id(obj))
        self.val = None
        
class A:
    attr = Attribute(name='attr')
    #attr = 'DEMO'
 
a = A()
print(id(a))
a.attr = 42
print(a.attr)
#del a.attr

b = A()
print(id(b))
b.attr = 43
print(a.attr)
print(b.attr)
#del b.attr

# a1 = A()
# a.attr = 44
# a1.attr = 45
# print(a.attr)
# print(a1.attr)
# del a1.attr

In [ ]:
# a.attr1 = '42'
a.attr2 = 42

In [35]:
class TypeCheckerMixin:

    def __setattr__(self, name, value):
        obj_attrs = vars(self)
        cls_attrs = vars(type(self))
        if name in cls_attrs:
            type_, default = cls_attrs[name]
            if isinstance(value, type_):
                self.__dict__[name] = value
            else:
                raise ValueError('Invalid type')

class A(TypeCheckerMixin):
    attr1 = (int, 0)
    attr2 = (str, '')
    attr3 = (bool, False)

a = A()

a.attr1 = 42
#a.attr1 = '42'

# But what if we want to check range for ints, regex amtch for strings, isclose() for floats?

In [34]:

class Descriptor:
    def __init__(self, name=None, default=None):
        self.name = name
        self.default = default

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

    def __get__(self, instance, objtype):
        if self.name not in instance.__dict__:
            instance.__dict__[self.name] = self.default
        return instance.__dict__[self.name]

    def __delete__(self, instance):
        raise AttributeError("Can't delete")


class Typed(Descriptor):
    type_ = object
    extra_methods = []
    def __set__(self, instance, value):
        if not isinstance(value, self.type_):
            raise TypeError('Expected %s' % self.type_)
        super().__set__(instance, value)


# Specialized types
class Numeric(Typed):
    extra_methods = ['gt', 'gte']

    def gt(instance_value, value):
        return instance_value > value

    def gte(instance_value, value):
        return instance_value >= value

class Integer(Numeric):
    type_ = int

class Float(Numeric):
    type_ = float
    extra_methods = Numeric.extra_methods + ['isclose']

    def isclose(instance_value, value):
        import math
        return math.isclose(instance_value, value)

class String(Typed):
    type_ = str
    extra_methods = ['startswith', 'endswith', 'contains']

    def startswith(instance_value, value):
        return instance_value.startswith(value)

    def endswith(instance_value, value):
        return instance_value.endswith(value)

    def contains(instance_value, value):
        return value in instance_value

In [36]:
class A:
    attr1 = Integer(name='attr1')

a = A()
a.attr1 = 32
# a.attr1 = '32'

In [37]:
# Value checking
class Positive(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)


# More specialized types
class PosInteger(Integer, Positive):
    pass


class PosFloat(Float, Positive):
    pass


# Length checking
class Sized(Descriptor):
    def __init__(self, *args, maxlen, **kwargs):
        self.maxlen = maxlen
        super().__init__(*args, **kwargs)

    def __set__(self, instance, value):
        if len(value) > self.maxlen:
            raise ValueError('Too big')
        super().__set__(instance, value)


class SizedString(String, Sized):
    pass


# Pattern matching
class Regex(Descriptor):
    def __init__(self, *args, pattern, **kwargs):
        self.pattern = re.compile(pattern)
        super().__init__(*args, **kwargs)

    def __set__(self, instance, value):
        if not self.pattern.match(value):
            raise ValueError('Invalid string')
        super().__set__(instance, value)


class SizedRegexString(SizedString, Regex):
    pass


In [38]:
class A:
    attr1 = PosInteger(default=42)
    attr2 = PosFloat()
    attr3 = SizedRegexString(maxlen=11, pattern='\d{3}-\d{7}')

a = A()
print(a.attr1)
a.attr1 = 32
print(a.attr1)
a.attr2 = 0.1
a.attr3 = '067-9372129'

a1 = A()
a1.attr1 = 5
print(a1.attr1)

print(id(a.attr1))
print(id(a1.attr1))

print(PosInteger.mro())

42
32
5
140073262912624
9752288
[<class '__main__.PosInteger'>, <class '__main__.Integer'>, <class '__main__.Numeric'>, <class '__main__.Typed'>, <class '__main__.Positive'>, <class '__main__.Descriptor'>, <class 'object'>]


### Metaclasses


Let's debug our code

In [39]:
def debug(func):
    '''
    A simple debugging decorator
    '''
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'{msg} run took {time.time()-start} ms')
        return result
    return wrapper

In [40]:
@debug
def foo():
    time.sleep(1)
    
foo()

foo run took 1.0007898807525635 ms


In [41]:
class A():
    
    @debug
    def foo(self):
        time.sleep(random.random())

a = A()
a.foo()

A.foo run took 0.3248271942138672 ms


In [42]:
class A():
    
    @debug
    def foo(self):
        time.sleep(random.random())
        
    @debug
    def bar(self):
        time.sleep(random.random())
        
    @debug
    def baz(self):
        time.sleep(random.random())
        
a = A()
a.foo()
a.bar()
a.baz()

A.foo run took 0.1924147605895996 ms
A.bar run took 0.6133582592010498 ms
A.baz run took 0.8557188510894775 ms


In [43]:
def debugmethods(cls):
    '''
    Apply a decorator to all callable methods of a class
    '''
    for name, val in vars(cls).items(): # cls.__dict__
        if callable(val):
            setattr(cls, name, debug(val))

    setattr(cls, 'xxx', 42)

    return cls

# A.foo = debug(A.foo)

# # A.bar = debug(A.bar)
# # setattr(A, 'bar', debug(A.bar))

# A.baz = debug(A.baz)
# help(callable)

In [44]:
@debugmethods
class A():
    
    def foo(self):
        time.sleep(random.random())
        
    def bar(self):
        time.sleep(random.random())
        
    def baz(self):
        time.sleep(random.random())
        
a = A()
a.foo()
a.bar()
a.baz()
print(a.xxx)

A.foo run took 0.8580012321472168 ms
A.bar run took 0.4554758071899414 ms
A.baz run took 0.2630341053009033 ms
42


In [45]:
class B(A):
    def foo_b(self):
        time.sleep(random.random())
        
    def bar_b(self):
        time.sleep(random.random())
        
    def baz_b(self):
        time.sleep(random.random())
        
b = B()
b.foo_b()
b.bar_b()
b.baz_b()


Let's use metaclasses to entire eirarchy with debug decorators

In [46]:
A = type('A', (object,), {'attr1': 42, 'attr2': 'abc'})

print(type(A), id(A))
print(A.__dict__)
a = A()
print(type(a))
print(a.attr1)
print(a.attr2)
a.attr1 = 43
print(a.attr1)

<class 'type'> 21345136
{'attr1': 42, 'attr2': 'abc', '__module__': '__main__', '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
<class '__main__.A'>
42
abc
43


In [47]:
class A:
    attr1 = 42
    attr2 = 'abc'
    
print(type(A), id(A))
print(A.__dict__)
a = A()
print(type(a))
print(a.attr1)
print(a.attr2)
a.attr1 = 43
print(a.attr1)

<class 'type'> 21261760
{'__module__': '__main__', 'attr1': 42, 'attr2': 'abc', '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
<class '__main__.A'>
42
abc
43


In [48]:
A = type('A',
         (object,),
         {'attr1': 42,
          'attr2': 'abc',
          'foo': lambda self: self.attr1+1}
        )
a = A()
a.foo()

43

<img src="https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/instance-of.png">

In [ ]:
print(type(a))
print(a.__class__)

In [ ]:
print(type(A))
print(A.__class__)

# not parent, but creator. Compare with
print(A.__bases__)

In [ ]:
print(type(type))
print(type.__class__)

# not parent, but creator. Compare with
print(type.__bases__)

In [ ]:
print(type(type(type(type(type(type))))))
print(type.__class__)

In [ ]:
class A(metaclass=type):
    def foo(self):
        print('foo')
        
a = A()
a.foo()
# print(a.bar)

In [49]:
class mytype(type):
    '''
    Metaclass default implementation
    '''
    def __new__(metacls, clsname, bases=None, clsdict=None):
        cls = super().__new__(metacls, clsname, bases, clsdict)
        cls.bar = 42
        setattr(cls, 'objects', 'XYZ')
        return cls

    # def __init__(cls, name, bases, dct):
    #     super().__init__(name, bases, dct)

In [50]:
class A(metaclass=mytype):
    def foo(self):
        print('foo')

class B(A):
    pass

a = A()
a.foo()
print(a.bar)


b = B()
b.foo()
print(b.bar)
print(b.objects)

foo
42
foo
42
XYZ


In [54]:
class ModelMeta(type):

    def __new__(metacls, clsname, bases=None, clsdict=None):
        cls = super().__new__(metacls, clsname, bases, clsdict)
        extra_attrs = []
        for attr_name, attr_value in cls.__dict__.items():
            if isinstance(attr_value, Typed):
                extra_attrs += [
                    (attr_name, extra_method, getattr(attr_value.__class__, extra_method))
                    for extra_method in attr_value.extra_methods
                ]

        for attr, extra, func in extra_attrs:
            setattr(
                cls,
                f'{attr}__{extra}',
                lambda self, value, attr=attr, func=func: func(getattr(self, attr), value)
            )

        return cls

class Employee(metaclass=ModelMeta):
    first_name = SizedString(name='first_name', default='John', maxlen=32)
    last_name = SizedString(name='last_name', maxlen=64)
    age = PosInteger(name='age', default=42)
    salary = PosFloat(name='salary')
    phone_number = SizedRegexString(name='phone_number', maxlen=11, pattern='\d{3}-\d{7}')

In [2]:

class Attribute:

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print('Retrieving', self.name, id(obj))
        return obj.__dict__[self.name]  # self.val

    def __set__(self, obj, val):
        print('Updating', self.name, id(obj))
        self.val = val
        obj.__dict__[self.name] = val

    def __delete__(self, obj):
        # print('Deleting', self.name, id(obj))
        self.val = None


class A:
    attr = Attribute(name='attr')
    # attr = 'DEMO'
    
a = A()
a.attr=10

Updating attr 140452546327024


In [52]:
emp = Employee()
print(emp.first_name)
print(emp.first_name__startswith('J'))
print(emp.age__gte(42))
emp.age = 10
print(emp.age__gt(42))

John
True
True
False


In [ ]:
class debugmeta(type):
    '''
    Metaclass that applies debugging to methods
    '''
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super().__new__(cls, clsname, bases, clsdict)
        clsobj = debugmethods(clsobj)
        return clsobj


In [ ]:
class A(metaclass=debugmeta):
    
    def foo(self):
        time.sleep(random.random())
        
    def bar(self):
        time.sleep(random.random())
        
    def baz(self):
        time.sleep(random.random())
        
a = A()
a.foo()
a.bar()
a.baz()

In [ ]:
class B(A):
    def foo_b(self):
        time.sleep(random.random())
        
    def bar_b(self):
        time.sleep(random.random())
        
    def baz_b(self):
        time.sleep(random.random())
        
b = B()
b.foo_b()
b.bar_b()
b.baz_b()